In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:

# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [3]:
if __name__ == '__main__':
    # Headers for request
    HEADERS = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0' , 'Accept-Language': 'en-US, en;q=0.5' })

    #URL Message
    URL = "https://www.amazon.com/s?k=playstation+5&crid=2YDRY3FVM91OE&sprefix=plays%2Caps%2C552&ref=nb_sb_ss_w_hit-vc-lth_playstation-5_k0_1_5"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [4]:
amazon_df

,title,price,rating,reviews,availability
0,PlayStation®5 console (slim) with EA SPORTS Co...,,4.9 out of 5 stars,23 ratings,In Stock
1,PlayStation 5 Digital Edition - CFI-1102B,,4.7 out of 5 stars,841 ratings,Only 11 left in stock - order soon.
2,Xbox Series X Console (Renewed),,4.1 out of 5 stars,928 ratings,In Stock
3,EA SPORTS College Football 25 - PlayStation 5,,4.6 out of 5 stars,757 ratings,In Stock
4,$150 PlayStation Store Gift Card [Digital Code],,4.6 out of 5 stars,"252,622 ratings",Available now
5,Madden NFL 25 - PlayStation 5,,4.6 out of 5 stars,163 ratings,Only 11 left in stock - order soon.
6,Seagate Game Drive for PS5 5TB External HDD - ...,,4.6 out of 5 stars,"23,863 ratings",In Stock
7,$250 PlayStation Store Gift Card [Digital Code],,4.7 out of 5 stars,"252,622 ratings",Available now
8,PlayStation Pulse Elite Wireless Headset,,4.0 out of 5 stars,"1,060 ratings",In Stock
9,PS5 / PS5 Slim Stand and Cooling Station with ...,,4.3 out of 5 stars,"8,242 ratings",In Stock
